In [ ]:
import rasterio
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv
import geoviews as gv
import rioxarray as rx
import os
import panel as pn
from holoviews import opts
hv.extension('bokeh', width=90)
pn.extension()
import xrspatial as xrs
from holoviews.streams import SingleTap
from decimal import Decimal
from math import isnan

In [ ]:
places={}
places['St.Thomas']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif"
places['St.Croix']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n18w065/USGS_1_n18w065.tif"
places['Cornell']='https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w076/USGS_1_n42w076.tif'

In [ ]:
maps = ['inferno','blues','kb','kgy','viridis']

# Creating variables which will store shared value

In [ ]:
data = xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif")[0,:,:].astype('float64')
data.attrs = xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif").attrs
data = data.where(data>0)
sloped = 0
aspects = 0
curvature = 0
maxheight = 0
minheight = 0
maxslope = 0
minslope = 0
maxaspect = 0
minaspect = 0
mincurve = 0
maxcurve = 0
test = ''
link = ''

In [ ]:
data.nbytes/1e6

In [ ]:
data

# Making useful helper functions

In [ ]:
def createlink(x,y):
    global link
    link = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/'
    north = "{:02d}".format(round(abs(y)))
    west =  "{:03d}".format(round(abs(x)))
    coordinates = 'n'+north+'w'+west
    finallink = link+coordinates+'/USGS_1_'+coordinates+'.tif'
    link = finallink
    return (finallink,coordinates) 

In [ ]:
def getmaxmin(dset):
    return (round(float(dset.min())),round(float(dset.max())))

In [ ]:
def getpercentage(orig,new):
    origcount = int((~np.isnan(orig.values)).sum())
    newcount = int((~np.isnan(new.values)).sum())
    return round((newcount/origcount)*100)

In [ ]:
def getregions(startpoint,endpoint):
    #NOTE: this functions treats all points as if they were soley positive
    #NOTE: this only works if the startpoint is below the end point. Remember to compensate for that in code.
    start = [floor(float(abs(startpoint[0]))),floor(float(abs(startpoint[1])))]
    end = [ceil(float(abs(endpoint[0]))),ceil(float(abs(endpoint[1])))]
    regions = []
    for horiz in range(start[0]+1,end[0]+1):
        for vert in range(start[1]+1,end[1]+1):
            regions.append((horiz,vert))
    return regions

In [ ]:
def getregiondata(points):
    #Function must be given the largest corner (both x and y) of all regions (treating the region as if all coordinates were positive)
    regions = []
    for p in points:
        try:
            place = createlink(p[0],p[1])[0]
            data = rx.open_rasterio(place, masked=True).squeeze().astype('float64')
            data.attrs = xr.open_rasterio(place).attrs
            regions.append(data)
        except:
            print('point '+str(p)+' is not avaliable')
    return regions

In [ ]:
def bounddata(startpoint,endpoint):
    regions = getregions(startpoint,endpoint)
    unprocessed = getregiondata(regions)
    large = combineregions(unprocessed)
    processed = large.sel(x=slice(endpoint[0],startpoint[0]),y=slice(endpoint[1],startpoint[1]))
    processed.attrs['res'] = (1,1)
    return processed

In [ ]:
def combineregions(data):
    processed=[]
    for tile in data:
        # Removes 'halo region' which extands beyond the degree square
        startpointx = round(float(tile.x[0]))
        startpointy = round(float(tile.y[0]))
        endpointx = round(float(tile.x[-1]))
        endpointy = round(float(tile.y[-1]))
        tile.name = 'box'
        tile = tile.sel(x=slice(startpointx,endpointx),y=slice(startpointy,endpointy))
        processed.append(tile)
    return xr.combine_by_coords([sq.to_dataset() for sq in processed]).to_array()[0]

In [ ]:
def ensurepointscorrect(startpoint,endpoint):
    #function returns a tuple of points where the first point is the bottom right corner and the second is the top left
    vertcheck = startpoint[1]<endpoint[1]
    horizcheck = startpoint[0]>endpoint[0]
    if vertcheck and horizcheck:
        return (startpoint,endpoint)
    else:
        dx = abs(startpoint[0]-endpoint[0])
        dy = abs(startpoint[1]-endpoint[1])
        starty = startpoint[1] if vertcheck else endpoint[1]
        startx = startpoint[0] if horizcheck else endpoint[0]
        endy = starty+dy
        endx = startx-dx
        return ((startx,starty),(endx,endy))

# Making widgets and Text Displays

In [ ]:
cmapwidget = pn.widgets.Select(name='Select Color Map', options=maps)
heightquantileslider = pn.widgets.IntSlider(name='Height Quantile Slider', start=0, end=32, value=7)
heightrangeslider = pn.widgets.RangeSlider(step=1,start = minheight, end = maxheight, value=(minheight,maxheight))
sloperangeslider = pn.widgets.RangeSlider(start = minslope, end=maxslope,step=1,value=(minslope,maxslope))
aspectrangeslider = pn.widgets.RangeSlider(start = minaspect, end=maxaspect,step=1,value=(minaspect,maxaspect))
curvaturerangeslider = pn.widgets.RangeSlider(start = mincurve, end=maxcurve,step=1,value=(mincurve,maxcurve))
selectlatp1 = pn.widgets.TextInput(name='selectlatp1', value='')
selectlonp1 = pn.widgets.TextInput(name='selectlonp1', value='')
selectlatp2 = pn.widgets.TextInput(name='selectlatp2', value='')
selectlonp2 = pn.widgets.TextInput(name='selectlatp2', value='')

In [ ]:
heightp = pn.pane.Markdown()
heightpercentpanel = pn.Column(heightp,css_classes=['panel-widget-box'],margin=25)
slopep = pn.pane.Markdown()
slopepercentpanel = pn.Column(slopep,css_classes=['panel-widget-box'],margin=25)
aspectp = pn.pane.Markdown()
aspectn = pn.pane.Markdown('# Aspect is the direction in which the land is pointing. The slider can be from 0-360 where 0 is absolute North')
aspectpercentpanel = pn.Column(aspectp,css_classes=['panel-widget-box'],margin=25)
aspectnote = pn.Column(aspectn,css_classes=['panel-widget-box'],margin=25)
curvep = pn.pane.Markdown()
curvaturepercentpanel = pn.Column(curvep,css_classes=['panel-widget-box'],margin=25)

# Creating the displays for each tab

In [ ]:
tiles = gv.tile_sources.EsriImagery().opts(width=600, height=550).redim.range(Latitude=(-50,50), Longitude=(-50,50))
#tiles = gv.tile_sources.EsriImagery().opts(width=600, height=550).opts(xlim=(-50, 50), ylim=(-50, 50))

tap_stream1 = SingleTap(source=tiles, x=10, y=0)
tap_stream2 = SingleTap(source=tiles, x=10, y=0)
@pn.depends(tap_stream1.param.x, tap_stream1.param.y)
def lat_lon(x,y):
    return f"### Lat: {y:.4f} Lon: {x:.4f}"

def interactive_click(x, y):
    return gv.Points([(x,y)])
click1 = hv.DynamicMap(interactive_click, streams=[tap_stream1])
click2 = hv.DynamicMap(interactive_click, streams=[tap_stream2])

In [ ]:
@pn.depends(colormap = cmapwidget, x=tap_stream1.param.x, y=tap_stream1.param.y)
def clicklocation(colormap,x,y):
    global data
    global test
    global sloped
    global aspects
    global curvature
    try:
        place = createlink(x,y)[0]
        data = rx.open_rasterio(place, masked=True).squeeze().astype('float64')
        data.attrs = xr.open_rasterio(place).attrs
        data.attrs['res'] = (1,1)
        data = data.where(data>0)
        test = 'All good!'
    except:
        test = 'There was an error'
    sloped = xrs.slope(data)
    aspects = xrs.aspect(data)
    curvature = xrs.curvature(data)
    (heightrangeslider.start,heightrangeslider.end)=getmaxmin(data)
    heightrangeslider.value = getmaxmin(data)
    (sloperangeslider.start,sloperangeslider.end)=getmaxmin(data)
    sloperangeslider.value = getmaxmin(data)
    (aspectrangeslider.start,aspectrangeslider.end)=getmaxmin(data)
    aspectrangeslider.value = getmaxmin(data)
    (curvaturerangeslider.start,curvaturerangeslider.end)=getmaxmin(data)
    curvaturerangeslider.value = getmaxmin(data)

In [ ]:
@pn.depends(colormap = cmapwidget,p1x = selectlonp1, p1y =selectlatp1, p2x = selectlonp2, p2y = selectlatp2)
def inputlocation(colormap,p1x,p1y,p2x,p2y):
    global data
    global test
    global sloped
    global aspects
    global curvature
    try:
        points = ensurepointscorrect((p1x,p1y),(p2x,p2y))
        data = bounddata(points[0],points[1])
        data = data.where(data>0)
        test = 'All good!'
    except:
        test = 'There was an error'
    sloped = xrs.slope(data)
    aspects = xrs.aspect(data)
    curvature = xrs.curvature(data)
    (heightrangeslider.start,heightrangeslider.end)=getmaxmin(data)
    heightrangeslider.value = getmaxmin(data)
    (sloperangeslider.start,sloperangeslider.end)=getmaxmin(data)
    sloperangeslider.value = getmaxmin(data)
    (aspectrangeslider.start,aspectrangeslider.end)=getmaxmin(data)
    aspectrangeslider.value = getmaxmin(data)
    (curvaturerangeslider.start,curvaturerangeslider.end)=getmaxmin(data)
    curvaturerangeslider.value = getmaxmin(data)

In [ ]:
@pn.depends(x=tap_stream1.param.x, y=tap_stream1.param.y)
def shaded(x,y):
    global data
    shade = xrs.hillshade(data)
    return shade.hvplot.image(x = 'x', y = 'y', rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget, alpha = 20)

In [ ]:
@pn.depends(x=tap_stream1.param.x, y=tap_stream1.param.y, level = heightquantileslider)
def quantile(x,y, level):
    global data
    quant = xrs.quantile(data,k=level,ignore_vals=(0,))
    return quant.hvplot.image(x = 'x', y = 'y',rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget, alpha = 20)

In [ ]:
@pn.depends(x=tap_stream1.param.x, y=tap_stream1.param.y, border = heightrangeslider)
def heightrange(x,y, border):
    global data
    manipulate = data.where(np.logical_and(data>border[0],data<border[1]))
    heightp.object = '# Your selected range is '+ str(getpercentage(data,manipulate))+'% of this region'
    shade = xrs.hillshade(manipulate)    
    return shade.hvplot.image(x = 'x', y = 'y',rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

In [ ]:
@pn.depends(x=tap_stream1.param.x, y=tap_stream1.param.y, border = sloperangeslider)
def slope(x,y, border):
    global sloped
    manipulate = sloped.where(np.logical_and(sloped>border[0],sloped<border[1]))
    slopep.object = '# Your selected range is '+ str(getpercentage(sloped,manipulate))+'% of this region'
    return manipulate.hvplot.image(x = 'x', y = 'y',rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

In [ ]:
@pn.depends(x=tap_stream1.param.x, y=tap_stream1.param.y, border = aspectrangeslider)
def aspect(x,y, border):
    global aspect
    manipulate = aspects.where(np.logical_and(aspects>border[0],aspects<border[1]))
    aspectp.object = '# Your selected range is '+ str(getpercentage(aspects,manipulate))+'% of this region'
    return manipulate.hvplot.image(x = 'x', y = 'y',rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

In [ ]:
@pn.depends(x=tap_stream1.param.x, y=tap_stream1.param.y, border = curvaturerangeslider)
def curvature(x,y, border):
    global curvature
    manipulate = curvature.where(np.logical_and(curvature>border[0],curvature<border[1]))
    curvep.object = '# Your selected range is '+ str(getpercentage(curvature,manipulate))+'% of this region'
    return curvature.hvplot.image(x = 'x', y = 'y',rasterize = True, geo = True, tiles = 'OSM', cmap = cmapwidget)

# Creating the canvas for each tab

In [ ]:
selector1 = pn.Column(pn.panel(lat_lon, width=500), (tiles * click1))
selector2 = pn.Column(pn.panel(lat_lon, width=500), (tiles * click2))
clicker = pn.Column(clicklocation,selector1,cmapwidget)
inputlocation = pn.Column(inputlocation,selector2,cmapwidget)
quantileview = pn.Row(quantile, heightquantileslider)
heightrangeview = pn.Row(heightrange, pn.Column(heightrangeslider,heightpercentpanel))
slopeview = pn.Row(slope, pn.Column(sloperangeslider,slopepercentpanel))
aspectview = pn.Row(aspect, pn.Column(aspectrangeslider,aspectpercentpanel),aspectnote)
curvatureview = pn.Row(curvature, pn.Column(curvaturerangeslider,curvaturepercentpanel))

In [ ]:
option = pn.Tabs(('Location Clicker',clicker),
                 ('Input Location',inputlocation)
                )

In [ ]:
dashboard = pn.Tabs(('Options',option), 
                    ("Shaded View", shaded),
                    ('Height Range', heightrangeview),
                    ('Height Quantile', quantileview),
                    ('Aspect View', aspectview),
                    ('Slope View', slopeview),
                    ('Curvature View', curvatureview),
                    dynamic = True
                   )

In [ ]:
dashboard.servable()

In [ ]:
test

In [ ]:
link